In [27]:
import squarify
import pandas as pd

In [28]:
df = pd.DataFrame({'group':['a','a','b','b'], 'value':[200,100,150,150]})
df = df.sort_values('group')
grp = df.groupby('group')['value'].sum()

In [29]:
x = 0.
y = 0.
width = 100.
height = 100.

values = grp.values

normed = squarify.normalize_sizes(values, width, height)
rects = squarify.squarify(normed, x, y, width, height)

In [30]:
rects

[{'dx': 100.0, 'dy': 50.0, 'x': 0.0, 'y': 0.0},
 {'dx': 100.0, 'dy': 50.0, 'x': 0.0, 'y': 50.0}]

In [31]:
subrects = []
for group, rect in zip(set(df['group']), rects):
    dff = df[df['group'] == group]['value']
    
    x = rect['x']
    y = rect['y']
    width = rect['dx']
    height = rect['dy']

    values = dff.values

    normed = squarify.normalize_sizes(values, width, height)
    rectgroup = squarify.padded_squarify(normed, x, y, width, height)
    
    subrects += rectgroup

In [32]:
subrects

[{'dx': 48.0, 'dy': 48.0, 'x': 1.0, 'y': 1.0},
 {'dx': 48.0, 'dy': 48.0, 'x': 51.0, 'y': 1.0},
 {'dx': 64.66666666666667, 'dy': 48.0, 'x': 1.0, 'y': 51.0},
 {'dx': 31.33333333333333,
  'dy': 48.00000000000001,
  'x': 67.66666666666667,
  'y': 51.0}]

In [33]:
def plotly_squares(rects, values):
    import plotly
    import plotly.graph_objs as go
    plotly.offline.init_notebook_mode(connected=True)
    
    color_brewer = ['rgb(166,206,227)','rgb(31,120,180)','rgb(178,223,138)',
                'rgb(51,160,44)','rgb(251,154,153)','rgb(227,26,28)']
    
    shapes = []
    annotations = []
    counter = 0

    for r in rects:
        shapes.append( 
            dict(
                type = 'rect', 
                x0 = r['x'], 
                y0 = r['y'], 
                x1 = r['x']+r['dx'], 
                y1 = r['y']+r['dy'],
                line = dict( width = 2 ),
                fillcolor = color_brewer[counter]
            ) 
        )
        
        annotations.append(
            dict(
                x = r['x']+(r['dx']/2),
                y = r['y']+(r['dy']/2),
                text = values[counter],
                showarrow = False
            )
        )
        counter = counter + 1
        if counter >= len(color_brewer):
            counter = 0

    # For hover text
    trace0 = go.Scatter(
        x = [ r['x']+(r['dx']/2) for r in rects ], 
        y = [ r['y']+(r['dy']/2) for r in rects ],
        text = [ str(v) for v in values ], 
        mode = 'text',
    )

    layout = dict(
        height=700, 
        width=700,
        xaxis=dict(showgrid=False,zeroline=False),
        yaxis=dict(showgrid=False,zeroline=False),
        shapes=shapes,
        annotations=annotations,
        hovermode='closest'
    )
    

    figure = dict(data=[trace0], layout=layout)

    plotly.offline.iplot(figure, filename='squarify-treemap')

In [34]:
plotly_squares(rects, values = grp.values)

In [35]:
plotly_squares(subrects, values = df['value'].values)